In [1]:
import pandas as pd
import argparse
import logging
import json
import os
import pathlib
import fiona
import numpy as np
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from geopandas.tools import geocode
from shapely.geometry import Point
import requests
from zipfile import ZipFile
from tqdm import trange, tqdm

In [4]:
AVG_KMPERYEAR_AUTOMOBILI = 12730 #REF
AVG_KMPERYEAR_AUTOMOBILI_HYB = 12730 #REF
AVG_KMPERYEAR_AUTOVEICOLILEGERI = 14329 #REF
AVG_KMPERYEAR_MOTOVEICOLI = 10000 #REF
AVG_KMPERYEAR_MOTOLEGGERE = 5000 #REF

AVG_LPERKM_AUTOMOBILI = 0.093
AVG_LPERKM_AUTOMOBILI_HYB = 0.065
AVG_LPERKM_AUTOVEICOLILEGERI = 0.131
AVG_LPERKM_MOTOVEICOLI = 0.04
AVG_LPERKM_MOTOLEGGERE = 0.018

pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MOBILITY_DIRECTORY = DATA_DIRECTORY /'mobility'

In [45]:
mobility_file = "TI_mobility_stats.csv"

mobility_num = pd.read_csv(MOBILITY_DIRECTORY/mobility_file).set_index("District")
#mobility_num = pd.read_csv(MOBILITY_DIRECTORY/mobility_file)

Automobili = mobility_num["Automobili"]
Automobili_pkm = Automobili*AVG_KMPERYEAR_AUTOMOBILI

Motoveicoli = mobility_num["Motoveicoli"]
Motoveicoli_pkm = Motoveicoli*AVG_KMPERYEAR_MOTOVEICOLI

Motoleggere = mobility_num["Motoveicoli"]
Motoleggere_pkm = Motoveicoli*AVG_KMPERYEAR_MOTOLEGGERE


In [41]:
Automobili_pkm

District
Mendrisio       427295180
Lugano         1225899000
Locarno         486540600
Vallemaggia      47177380
Bellinzona      435926120
Riviera          85265540
Blenio           47941180
Leventina        74992430
Name: Automobili, dtype: int64

In [32]:
TI_carburant_file = "TI_anno_carburante.csv"
Automobility_carburanti = pd.read_csv(MOBILITY_DIRECTORY/TI_carburant_file).set_index("Carburant")

In [33]:
Automobility_carburanti_2020 = Automobility_carburanti['2020']
Automobility_sum_2020 = Automobility_carburanti_2020.sum()

In [34]:
print(Automobility_carburanti_2020, "sum =", Automobility_sum_2020)

Carburant
Essence                                     148394
Diesel                                       63454
Essence-electrique: hybride normal            6274
Essence-electrique: hybride rechargeable      1305
Diesel-electrique: hybride normal              610
Diesel-electrique: hybride rechargeable         38
Electrique                                    1689
Hydrogene                                        0
Gaz (monovalent et bivalent)                   558
Autre                                           69
Name: 2020, dtype: int64 sum = 222391


In [35]:
Automobility_carburanti_2020_share = Automobility_carburanti_2020/Automobility_sum_2020

In [36]:
Automobility_carburanti_2020_share.to_csv("test.csv")
Automobility_carburanti_2020_share

Carburant
Essence                                     0.667266
Diesel                                      0.285326
Essence-electrique: hybride normal          0.028212
Essence-electrique: hybride rechargeable    0.005868
Diesel-electrique: hybride normal           0.002743
Diesel-electrique: hybride rechargeable     0.000171
Electrique                                  0.007595
Hydrogene                                   0.000000
Gaz (monovalent et bivalent)                0.002509
Autre                                       0.000310
Name: 2020, dtype: float64

In [56]:
Auto_pkm_carburant = pd.DataFrame()

for dis in range(len(Automobili_pkm)):
    pkm_carb = Automobili_pkm.iloc[dis]*Automobility_carburanti_2020_share
    Auto_pkm_carburant=  pd.concat([Auto_pkm_carburant, pkm_carb], axis=1)

Auto_pkm_carburant = Auto_pkm_carburant.T
Auto_pkm_carburant["District"] = Automobili_pkm.index
Auto_pkm_carburant = Auto_pkm_carburant.set_index("District", drop = True)
Auto_pkm_carburant.to_csv("private_mobility_pkm_per_carb.csv")